In [61]:
import numpy as np


class Edge:
    def __init__(self, from_node, to_node, weight) -> None:
        self.from_node = from_node
        self.to_node = to_node
        self.weight = weight


def sigmoid(x):
    return 1 / (1 + pow(2.71828, -x))


def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))


class Node:
    def __init__(self, name):
        self.name = name
        self.children = []
        self.parents = {}
        self.bias = 0
        self.activation = 0

    # def add_child(self, name, weight):
    #     self.children.append(Edge(self.name, name, weight))

    def add_parent(self, name, weight):
        self.parents[name] = weight

    def forward(self, x):
        if self.parents:
            pre_sig = (
                sum(
                    [
                        self.parents[parent_name] * x[parent_name]
                        for parent_name in self.parents.keys()
                    ]
                )
                + self.bias
            )
            self.activation = sigmoid(pre_sig)

        else:
            self.activation = x[self.name]

        return self.activation

    def backward(self, x):
        if self.parents:
            return sum([child.weight * x[child.to_node] for child in self.children])
        else:
            return x[self.name]


# input nodes:
nodes = {}
node_names = [f"x{i}" if i < 3 else f"a{i}" for i in range(6)]
for name in node_names:
    nodes[name] = Node(name)

print([node.name for name, node in nodes.items()])

edges = [
    ("x0", "a3", 1),
    ("x0", "a4", -6),
    ("x0", "a5", -3.93),
    ("x1", "a3", 3),
    ("x1", "a4", 6),
    ("x2", "a3", 4),
    ("x2", "a4", 5),
    ("a3", "a5", 2),
    ("a4", "a5", 4),
]

for from_node, to_node, weight in edges:
    # nodes[from_node].add_child(to_node, weight)
    nodes[to_node].add_parent(from_node, weight)


def forward(x):
    for n, node in nodes.items():
        x[node.name] = node.forward(x)
        # print(f"{node.name} -> {x[node.name]}")
    return x


def backprop(activations, y_target, learn_rate=0.1):
    l_err = y_target - activations["a5"]
    # print(f"l_err: {l_err}")

    for n, node in reversed(nodes.items()):
        for name, weight in node.parents.items():
            z = activations[n]
            dw = l_err * sigmoid_prime(z) * nodes[name].activation
            nodes[n].parents[name] += learn_rate * dw

            # print(
            #     f"{l_err}, updating {name} -> {n} with {dw} {nodes[n].parents[name]}, {nodes[name].activation}"
            # )

            # nodes[n].bias += learn_rate * l_err * sigmoid_prime(z)


x = {
    "x0": 1,
    "x1": 1,
    "x2": 0,
}
# nodes["x0"].bias = 1

print(forward(x)["a5"])

y_target = 0.4

for i in range(1000):
    activations = forward(x)
    # print("output", activations["a5"])
    backprop(activations, y_target, learn_rate=0.1)

activations = forward(x)
print("output", activations["a5"])

['x0', 'x1', 'x2', 'a3', 'a4', 'a5']
0.5085060448109715
output 0.4000000000029003
